In [1]:
import pandas as pd
import heapq

In [2]:
df = pd.read_csv('Road_Distance.csv')
df.head()

,Distance in Kilometres,Ahmedabad,Bangalore,Bhubaneshwar,Bombay,Calcutta,Chandigarh,Cochin,Delhi,Hyderabad,...,Jaipur,Kanpur,Lucknow,Madras,Nagpur,Nasik,Panjim,Patna,Pondicherry,Pune
0,Agartala,3305,3824,2286,3593,1863,2998,4304,2708,3330,...,2801,2281,2252,3493,2696,3365,3507,1681,3661,3442
1,Agra,878,1848,1578,1202,1300,448,2278,200,1246,...,230,290,369,2048,770,1005,1715,885,2210,1214
2,Ahmedabad,-,1490,1697,552,2068,1157,1845,911,1436,...,648,1168,1247,1821,965,504,1165,1656,1818,664
3,Allahabad,1251,1686,1090,1457,817,912,2216,650,1084,...,713,193,234,2011,608,1155,1419,402,1077,1364
4,Amritsar,1356,2496,2224,1849,1919,239,3163,445,1892,...,706,926,939,2688,1416,1665,2237,1531,2856,1862


In [3]:
for i, value in enumerate(df.columns[1:]):
    df[value].replace(['-'], 0, inplace=True)

# Graph Building

In [4]:
def graph_building(df):
    map = {}
    cities = list(df.iloc[:, 0])
    for i in range(len(cities)):
        for j in df.columns[1:]:
            temp = {j: int(df.iloc[i][j]) for j in df.columns[1:] if df.iloc[i][j] != 0}
        map[cities[i]] = temp
        
    map2 = {}
    cities = list(df.columns[1:])
    for i in range(len(cities)):
        for j in df.index:
            temp = {df.iloc[j][0]: int(df.iloc[j][i+1]) for j in df.index if df.iloc[j][i+1] != 0}
        map2[cities[i]] = temp

    map3 = {}
    for key in map.keys():
        if key in map2.keys():
            map3[key] = {**map[key], **map2[key]}
        else:
            map3[key] = map[key]
            
    for i in map3.keys():
        for j in map3[i].keys():
            map3[i][j] = int(map3[i][j])
    
    return map3, map

In [5]:
map3, map = graph_building(df)

# Uniform Cost Search

In [6]:
import heapq

def uniform_cost_search(graph, start, goal):
    priority_queue = [(0, start, [])]
    visited = set()

    while priority_queue:
        cost, current_node, path = heapq.heappop(priority_queue)

        if current_node == goal:
            return cost, path + [current_node]

        visited.add(current_node)

        for neighbor, neighbor_cost in graph[current_node].items():
            if neighbor not in visited:
                new_cost = cost + int(neighbor_cost)
                new_path = path + [current_node]
                heapq.heappush(priority_queue, (new_cost, neighbor, new_path))

    return None, []


In [32]:
start_city = input("Enter the start city: ")
goal_city = input("Enter the goal city: ")

result, path = uniform_cost_search(map3, start_city, goal_city)

if result is not None:
    print(f"Shortest path cost from {start_city} to {goal_city} is {result}")
    print(f"Shortest path is: {path}")
else:
    print(f"No path found from {start_city} to {goal_city}")

Shortest path cost from Agartala to Madurai is 3493
Shortest path is: ['Agartala', 'Patna', 'Allahabad', 'Pondicherry', 'Madurai']


# A* Search

In [75]:
def heuristic(node, goal, graph, h_n, flag):
    if flag:
        if goal == node:
            return 0
        else:
            return dfs_path(h_n, node, goal)[1] // 100
    else:
        if goal == node:
            return 0
        elif goal in graph[node]:
            return max(graph[node][goal], graph[node][goal])
        else:
            return max(min([graph[node][i] + graph[goal][i] for i in graph[node]]), min([graph[node][i] + graph[goal][i] for i in graph[node]]))
        
def dfs_path(graph, start, end, path=[], cost=0):

    path = path + [start]
    if start == end:
        return path, cost
    if start not in graph:
        return None, 0
    for vertex, weight in graph[start]:
        if vertex not in path:
            newpath, newcost = dfs_path(graph, vertex, end, path, cost + weight)
            if newpath:
                return newpath, newcost
    return None, 0
    
def prim_mst(graph):

    mst = []
    visited = set()
    edges = [(u, v, w) for u in graph for v, w in graph[u].items()]
    edges.sort(key=lambda x: x[2])

    start_vertex = list(graph.keys())[0]
    visited.add(start_vertex)

    while len(visited) < len(graph):
        candidate_edges = [(u, v, w) for u, v, w in edges if u in visited and v not in visited]
        u, v, w = min(candidate_edges, key=lambda x: x[2])
        mst.append((u, v, w))
        visited.add(v)
    
    return mst
    
def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

def astar2(graph, start, goal, flag=True):
    h_n = prim_mst(graph)
    open_set = [(0, start)]
    came_from = {}
    
    g_score = {city: float('inf') for city in graph}
    g_score[start] = 0
    
    f_score = {city: float('inf') for city in graph}
    f_score[start] = heuristic(start, goal, graph, h_n, flag)
    
    explored = set()
    
    all_cities = len(set(graph.keys()))

    while len(explored) != all_cities and open_set:
        _, current = heapq.heappop(open_set)
        explored.add(current)

        for neighbor in graph[current]:
            tentative_g_score = g_score[current] + graph[current][neighbor]
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal, graph, h_n, flag)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return g_score, came_from

In [76]:
start_city = input("Enter the start city: ")
goal_city = input("Enter the goal city: ")

g_score, came_from = astar2(map3, start_city, goal_city)

if goal_city in came_from:
    path = reconstruct_path(came_from, goal_city)
    print("Path from {} to {}: {}".format(start_city, goal_city, path))
    print("Total cost: {}".format(g_score[goal_city]))
else:
    print("No path found from {} to {}.".format(start_city, goal_city))

Path from Agartala to Madurai: ['Agartala', 'Patna', 'Allahabad', 'Pondicherry', 'Madurai']
Total cost: 3493


In [ ]:
while True:
    
    choice = int(input("Enter the choice of Algorithm: "))
    print("""
    1. Uniform Cost Search
    2. A* with Admissible
    3. A* with non-Admissible
    4. Exit
          """)
    
    if choice == 1:
        start_city = input("Enter the start city: ")
        goal_city = input("Enter the goal city: ")

        result, path = uniform_cost_search(map3, start_city, goal_city)

        if result is not None:
            print(f"Shortest path cost from {start_city} to {goal_city} is {result}")
            print(f"Shortest path is: {path}")
        else:
            print(f"No path found from {start_city} to {goal_city}")
    
    elif choice == 2:
        start_city = input("Enter the start city: ")
        goal_city = input("Enter the goal city: ")

        g_score, came_from = astar2(map3, start_city, goal_city, flag=True)

        if goal_city in came_from:
            path = reconstruct_path(came_from, goal_city)
            print("Path from {} to {}: {}".format(start_city, goal_city, path))
            print("Total cost: {}".format(g_score[goal_city]))
        else:
            print("No path found from {} to {}.".format(start_city, goal_city))
    
    elif choice == 3:
        start_city = input("Enter the start city: ")
        goal_city = input("Enter the goal city: ")

        g_score, came_from = astar2(map3, start_city, goal_city, flag=False)

        if goal_city in came_from:
            path = reconstruct_path(came_from, goal_city)
            print("Path from {} to {}: {}".format(start_city, goal_city, path))
            print("Total cost: {}".format(g_score[goal_city]))
        else:
            print("No path found from {} to {}.".format(start_city, goal_city))
            
    else:
        break